<h4>I used Claude, Anthropic's model to help with the code portion of this assignment.</h4>
<h4>I used Copilot for help understanding these concepts.</h4>

<h3>(1) Collect data to which applying an autoregression might make sense. Explain why autoregression might be a good fit for the data. Use an AR(1) and and an AR(optimal) model to predict the data and display the original data and predicted values in a time series chart. Check if the AIC and BIC criteria both result in the same number of lags for the AR(optimal) model.</h3>

In [7]:
# TODO Import python libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [8]:
# read in gold, and also convert everything into usd since it was indian ruppess.

df = pd.read_csv('../datasets/gold_price.csv')
df.head(5)
df.info()
# it only tracks the business days which makes sense I guess. keep that in mind for when I set the cadence or horizon I guess.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2848 entries, 0 to 2847
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2848 non-null   object 
 1   Price   2848 non-null   int64  
 2   Open    2848 non-null   int64  
 3   High    2848 non-null   int64  
 4   Low     2848 non-null   int64  
 5   Volume  2848 non-null   int64  
 6   Chg%    2848 non-null   float64
dtypes: float64(1), int64(5), object(1)
memory usage: 155.9+ KB


In [9]:
import pandas as pd

def convert_gold_prices_to_usd(df, usd_inr_rate=84.0):
    """
    Convert gold price DataFrame from INR to USD using a fixed exchange rate.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with columns: Date, Price, Open, High, Low, Volume, Chg%
    usd_inr_rate : float
        USD/INR exchange rate (default: 84.0, i.e., 1 USD = 84 INR)
    
    Returns:
    --------
    pandas DataFrame with prices converted to USD
    """
    # Create a copy to avoid modifying original
    df_usd = df.copy()
    
    # Convert Date to datetime if not already
    df_usd['Date'] = pd.to_datetime(df_usd['Date'])
    
    # Convert price columns from INR to USD
    price_columns = ['Price', 'Open', 'High', 'Low']
    for col in price_columns:
        df_usd[col] = (df_usd[col] / usd_inr_rate).round(2)
    
    # Display summary
    print(f"Conversion complete using rate: 1 USD = {usd_inr_rate} INR")
    print(f"Total records converted: {len(df_usd)}")
    print(f"\nPrice Statistics (USD):")
    print(f"  Mean: ${df_usd['Price'].mean():.2f}")
    print(f"  Min:  ${df_usd['Price'].min():.2f}")
    print(f"  Max:  ${df_usd['Price'].max():.2f}")
    print(f"\nFirst 5 rows:")
    print(df_usd[['Date', 'Price', 'Open', 'High', 'Low']].head())
    
    return df_usd

In [ ]:
# Assuming your DataFrame is named 'df'
df_usd = convert_gold_prices_to_usd(df, usd_inr_rate=84.0)

# For autoregression modeling:
df_usd = df_usd.sort_values('Date').reset_index(drop=True)
prices = df_usd['Price'].values # this is my target and the variable i'll be using lags with.

Conversion complete using rate: 1 USD = 84.0 INR
Total records converted: 2848

Price Statistics (USD):
  Mean: $490.74
  Min:  $292.20
  Max:  $943.54

First 5 rows:
        Date   Price    Open    High     Low
0 2025-01-06  918.44  920.35  923.12  911.25
1 2025-01-03  914.44  919.60  935.71  912.06
2 2025-01-02  914.87  914.87  914.87  914.87
3 2025-01-01  907.31  907.52  908.36  905.39
4 2024-12-31  907.52  900.95  916.31  899.67


In [12]:
df_usd.head(15)

,Date,Price,Open,High,Low,Volume,Chg%
0,2014-01-01,351.69,350.42,352.36,349.29,2930,0.25
1,2014-01-02,356.85,353.31,357.74,353.31,3140,1.47
2,2014-01-03,353.89,357.51,358.63,351.65,3050,-0.83
3,2014-01-04,348.56,348.56,348.56,348.56,0,-1.51
4,2014-01-06,346.65,348.81,349.94,345.85,24380,-0.55
5,2014-01-07,344.75,346.79,347.56,344.19,18710,-0.55
6,2014-01-08,344.45,344.24,345.58,343.10,18140,-0.09
7,2014-01-09,345.20,345.12,345.87,343.63,15130,0.22
8,2014-01-10,347.25,345.60,347.60,344.76,15810,0.59
9,2014-01-13,348.95,347.26,349.17,346.82,13780,0.49


In [ ]:
# i really only want price since I will use previous price dates for help with the predictions.


<h3>(2) Hide the end of the data set and forecast the series using the different models. Decide which model you would use and explain why.</h3>

<h3>(3) Add one other independent variable to the model and compare the forecasts of this updated model to the previous ones.</h3>